In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/train.csv")

In [3]:
df['ground_truth'][0]

"First calculate the volume of the aquarium by multiplying its length, width and height: 4 ft * 6 ft * 3 ft = 72 cubic ft\nThen figure out what proportion of the aquarium is full after the cat knocks it over: 1/2 * 1/2 = 1/4\nThen figure out what proportion of the aquarium is full after Nancy refills it: 3 * 1/4 = 3/4\nNow multiply the proportion of the aquarium that's full by the aquarium's volume to find out how much water is in it: 72 cubic ft * 3/4 = 54 cubic ft\n 54"

In [4]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message.content)

Hello! How can I help you today?


In [5]:
synthetic_data = []

In [6]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

client = OpenAI()

def generate_synthetic_correct_answer(question: str, correct_answer: str) -> str:
    system_prompt = f"""You're a synthetic data generator for Maths questions. Your job is to generate synthetic correct answer to a given question. Only return the correct answer to the question. Make sure there is variety in answering style mimicing answers given by different students unrelated and not at all same as the sample original answer.
    
    Example: 
    Question- {question}
    Correct Answer- {correct_answer}
    """

    user_prompt = f"For the following question - {question}. generate 1 correct synthetic correct answer."

    completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    )

    # print(completion.choices[0].message.content)
    return completion.choices[0].message.content

In [7]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

client = OpenAI()

def generate_synthetic_incorrect_answer(question: str, incorrect_answer: str, student_profile: str) -> str:
    system_prompt = f"""You're a helpful class teacher. Your job is to generate synthetic incorrect answer to a given question. Only return the incorrect answer to the question. Make sure you try to understand the psychology behind the student and understand where he might go wrong while generating synthetic wrong answer using the student profile: {student_profile}. You're a helpful class teacher. Your job is to generate synthetic correct answer to a given question. Only return the correct answer to the question. Make sure there is variety in answering style mimicing answers given by different students unrelated and not at all same as the sample original answer
    
    Example: 
    Question- {question}
    Student Profile: {student_profile}
    Incorrect Answer- {incorrect_answer}
    """

    user_prompt = f"For the following question - {question}. generate ONLY 1 synthetic incorrect answer."

    completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    )

    # print(completion.choices[0].message.content)
    return completion.choices[0].message.content

In [8]:
df['question'].unique()

array(["Nancy is filling an aquarium for her fish. She fills it halfway and goes to answer the door. While she's gone, her cat knocks the aquarium over and spills half the water in it. Then Nancy comes back and triples the amount of water in the aquarium. If the aquarium is 4 feet long, 6 feet wide, and 3 feet high, how many cubic feet of water are in the aquarium?",
       'John is very unfit and decides to work up to doing a push-up.  He trains 5 days a week for them and starts with wall push-ups.  He adds 1 rep a day and once he gets to 15 reps he will start training high elevation push-ups. and then low elevation push-ups, and finally floor push-ups.  How many weeks will it take him to get to floor push-ups?',
       'James has 20 pairs of red socks and half as many black socks.  He has twice as many white socks as red and black combined.  How many total socks does he have combined?',
       ...,
       "Half of Taylor's house guests like weak coffee and the other half like strong 

In [9]:
question_answer_dataset = []

for index, row in df.iterrows():
    question = row['question']

    correct_answers = [str(row['ground_truth'])]

    # Generate 2 more correct answers.
    for i in range(2):
        correct_answers.append(generate_synthetic_correct_answer(question, str(row['ground_truth'])))

    incorrect_answers = [str(row['student_incorrect_solution'])]

    for i in range(2):
        incorrect_answers.append(generate_synthetic_incorrect_answer(question, str(row['student_incorrect_solution']), row['student_profile']))
        
    question_answer_dataset.append({
        "question": question,
        "correct_answers": correct_answers, 
        "incorrect_answers": incorrect_answers
    })
    
    print(f"{index+1}/{df.shape[0]}")

1/2262
2/2262
3/2262
4/2262
5/2262
6/2262
7/2262
8/2262
9/2262
10/2262
11/2262


KeyboardInterrupt: 

In [36]:
correct_answers

["First calculate the volume of the aquarium by multiplying its length, width and height: 4 ft * 6 ft * 3 ft = 72 cubic ft\nThen figure out what proportion of the aquarium is full after the cat knocks it over: 1/2 * 1/2 = 1/4\nThen figure out what proportion of the aquarium is full after Nancy refills it: 3 * 1/4 = 3/4\nNow multiply the proportion of the aquarium that's full by the aquarium's volume to find out how much water is in it: 72 cubic ft * 3/4 = 54 cubic ft\n 54",
 'To find out how many cubic feet of water are in the aquarium, we first calculate the volume of the aquarium by multiplying its length, width, and height: 4 ft * 6 ft * 3 ft = 72 cubic ft. Then, after the cat knocks it over, the aquarium is 1/2 full. After Nancy refills it, the aquarium is 1/2 * 1/2 = 1/4 full. When Nancy triples the amount of water, it becomes 3/4 full. So, 72 cubic ft * 3/4 = 54 cubic ft. Therefore, there are 54 cubic feet of water in the aquarium.',
 "First, calculate the volume of the aquarium 

In [37]:
incorrect_answers

['The aquarium has a volume of 4 x 6 x 3 = 72 cubic feet.\nWhen Nancy fills it halfway, she fills it with 72/2 = 36 cubic feet of water.\nWhen the cat spills half of that, there are 36/2 = 18 cubic feet of water left.\nWhen Nancy triples that amount, she adds 18 x 3 = 54 cubic feet of water.\nThe aquarium now has 18 + 54 = 72 cubic feet of water.\n 72',
 'The aquarium has a volume of 4 x 6 x 3 = 72 cubic feet. \n\nNancy fills it halfway, so initially, she adds 72/2 = 36 cubic feet of water. \n\nWhen the cat spills half of that, there are 36/2 = 18 cubic feet of water remaining. \n\nThen, when Nancy triples that amount, she adds 18 x 3 = 54 cubic feet of water. \n\nFinally, adding the initial amount and the tripled amount, the aquarium will have 18 + 54 = 72 cubic feet of water. \n\nSo, the answer is 72 cubic feet.',
 'The aquarium has a volume of 4 x 6 x 3 = 72 cubic feet.\nWhen Nancy fills it halfway, she fills it with 72/2 = 36 cubic feet of water.\nWhen the cat spills half of that, 